# Sparsity analysis

## Dynamic analysis

In [1]:
import torch
from collections import deque
import os

import networkx as nx
import dgl

import matplotlib.pyplot as plt
import numpy as np

import polygraphs as pg
from polygraphs import graphs
from polygraphs import hyperparameters as hparams
from polygraphs import visualisations as viz
from polygraphs import ops


# Create a PolyGraph configuration
params = hparams.PolyGraphHyperParameters()

# Initial beliefs are random uniform between 0 and 1
params.init.kind = 'uniform'
# Chance that action B is better than action A
params.epsilon = 0.001

params.network.kind = 'complete'
params.network.size = 4

# Enable logging; print progress every 100 steps
params.logging.enabled = True
params.logging.interval = 100

# Take snapshots
params.snapshots.enabled = True
params.snapshots.interval = 100

params.simulation.steps = 0
params.simulation.repeats = 1

# Store results in directory
params.simulation.results = "data/test"

# Remove directory, if exists
if os.path.isdir(params.simulation.results):
    os.rmdir(params.simulation.results)

# Set seed
params.seed = 123456789

pg.random(params.seed)
_ = pg.simulate(params, op=ops.BalaGoyalOp)

Using backend: pytorch


[MON] step 0001 Ksteps/s   0.00 A/B 0.50/0.50
[MON] step 0100 Ksteps/s   0.43 A/B 0.50/0.50
[MON] step 0200 Ksteps/s   0.42 A/B 0.75/0.25
[MON] step 0300 Ksteps/s   0.37 A/B 0.50/0.50
[MON] step 0400 Ksteps/s   0.37 A/B 0.75/0.25
[MON] step 0500 Ksteps/s   0.36 A/B 0.50/0.50
[MON] step 0600 Ksteps/s   0.36 A/B 0.75/0.25
[MON] step 0700 Ksteps/s   0.36 A/B 0.75/0.25
[MON] step 0800 Ksteps/s   0.36 A/B 0.75/0.25
[MON] step 0900 Ksteps/s   0.36 A/B 0.75/0.25
[MON] step 1000 Ksteps/s   0.36 A/B 0.75/0.25
[MON] step 1100 Ksteps/s   0.36 A/B 0.50/0.50
[MON] step 1200 Ksteps/s   0.36 A/B 0.50/0.50
[MON] step 1300 Ksteps/s   0.37 A/B 0.50/0.50
[MON] step 1400 Ksteps/s   0.37 A/B 0.50/0.50
[MON] step 1500 Ksteps/s   0.38 A/B 0.50/0.50
[MON] step 1600 Ksteps/s   0.38 A/B 0.75/0.25
[MON] step 1700 Ksteps/s   0.39 A/B 0.75/0.25
[MON] step 1800 Ksteps/s   0.39 A/B 0.75/0.25
[MON] step 1900 Ksteps/s   0.39 A/B 0.75/0.25
[MON] step 2000 Ksteps/s   0.39 A/B 0.75/0.25
[MON] step 2100 Ksteps/s   0.39 A/

In [2]:
import h5py


def export(graph, path):
    """
    Export graph to Pajek format.
    """
    # Remove self-loops
    g = dgl.remove_self_loop(graph)
    graphx = nx.DiGraph(dgl.to_networkx(g))
    nx.write_pajek(graphx, path)


def postprocess(directory, id):
    """
    Post-process graph snapshots
    """
    # Resulting hashtable
    ht = {}
    graphs, _ = dgl.load_graphs(os.path.join(directory, f"{id}.bin"))
    graph = graphs[0]
    fp = h5py.File(os.path.join(directory, f"{id}.hd5"), "r")
    _keys = [int(key) for key in fp.keys()]
    _keys = sorted(_keys)
    for key in _keys:
        graph.ndata["beliefs"] = torch.tensor(fp[str(key)][:])
        dst = os.path.join(directory, f"export-{key}.pj")
        print(f"DBG> export to {dst}")
        export(graph, dst)


ht = postprocess("data/test", 1)

DBG> export to data/test/export-1.pj
DBG> export to data/test/export-100.pj
DBG> export to data/test/export-200.pj
DBG> export to data/test/export-300.pj
DBG> export to data/test/export-400.pj
DBG> export to data/test/export-500.pj
DBG> export to data/test/export-600.pj
DBG> export to data/test/export-700.pj
DBG> export to data/test/export-800.pj
DBG> export to data/test/export-900.pj
DBG> export to data/test/export-1000.pj
DBG> export to data/test/export-1100.pj
DBG> export to data/test/export-1200.pj
DBG> export to data/test/export-1300.pj
DBG> export to data/test/export-1400.pj
DBG> export to data/test/export-1500.pj
DBG> export to data/test/export-1600.pj
DBG> export to data/test/export-1700.pj
DBG> export to data/test/export-1800.pj
DBG> export to data/test/export-1900.pj
DBG> export to data/test/export-2000.pj
DBG> export to data/test/export-2100.pj
DBG> export to data/test/export-2200.pj
DBG> export to data/test/export-2228.pj


/Users/alex/Development/polygraphs/.venv/lib/python3.7/site-packages/networkx/readwrite/pajek.py:94: UserWarning: Edge attribute id is not processed. Non-string attribute.
  f"Edge attribute {k} is not processed. {('Empty attribute' if isinstance(v, str) else 'Non-string attribute')}."
